In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from IPython.display import display

# TODOs:
# automate tracefile generation and symbol analysis
# Get a native benchmark and non-native bench working

# Attestation Datasets
att_full1 = pd.read_csv("data/att_full1.csv")
att_full2 = pd.read_csv("data/att_full2.csv")
att_full3 = pd.read_csv("data/att_full3.csv")
att_full4 = pd.read_csv("data/att_full4.csv")
att_full5 = pd.read_csv("data/att_full5.csv")

att_trim1 = pd.read_csv("data/att_trim1.csv")
att_trim2 = pd.read_csv("data/att_trim2.csv")
att_trim3 = pd.read_csv("data/att_trim3.csv")
att_trim4 = pd.read_csv("data/att_trim4.csv")
att_trim5 = pd.read_csv("data/att_trim5.csv")

# Tiny-AES Datasets
aes1 = pd.read_csv("data/tinyaes1.csv")
aes2 = pd.read_csv("data/tinyaes2.csv")
aes3 = pd.read_csv("data/tinyaes3.csv")


In [2]:
class Boundary:
    """A boundary is defined by a start and end symbol along with a name."""
    def __init__(self, name, start, end):
        self.name = name
        self.start = start
        self.end = end

In [3]:
"""Given a set of boundaries (in no order) gather the number of cycles it took
to reach the end of each of the boundaries, by returning a pandas dataframe of
the boundaries, their labels, and the cycle cost. This should work in the case
of multiple enclaves being started, running a user/runtime, and then destroying.
"""

def get_indicies(bset, data):
    """Return the indicies of all the found possible boundaries.
    """
    start_symbols = [b.start for b in bset]
    end_symbols = [b.end for b in bset]
    indicies = []
    for i, row in data.iterrows():
        if row['symbol'] in start_symbols or row['symbol'] in end_symbols:
            if i not in indicies:
                indicies.append(i)
                
    return indicies

def get_cost_table(data):
    """Return table with cycle cost in place of the relative cycle number
    """
    data["end"] = data["symbol"].shift(-1)
    data["cost"] = -(data['cycle'] - data['cycle'].shift(-1))
    # drop the last row because this is cycle cost per instruction
    data = data.drop(['cycle'], axis=1)[:-1]
    return data

def get_symbol_cost_table(phase_df, data):
    """Given a phase cycles table, extract the start and end indicies for each phase, and break
    down those symbols into a pivot table from the original data table.
    """
    # get cost table from original data
    cost_data = get_cost_table(data)
    result = []
    for _, row in phase_df.iterrows():
        start = row['start_index']
        phase = row['phases']
        end = row['end_index']
        symbol_cost_summed = cost_data[start:end].groupby('symbol').sum()
        symbol_cost_summed.reset_index(inplace=True)
        symbol_data = symbol_cost_summed.pivot(columns='symbol', values='cost').bfill().iloc[[0],:]
        symbol_data['phase'] = phase
        result.append(symbol_data)
    # merge dataframes and index on phase
    final = pd.concat(result, ignore_index=True).set_index('phase')
    # sort columns by mean value within column
    return final.reindex(final.mean().sort_values().index, axis=1)

def get_phase_cycles_table(indicies, bset, data):
    """Given indicies of interest and the corresponding data
    return the cycle count of each phase, along with the 
    start and end indicies of each phase which can be used
    for further analysis
    """
    start_symbols = [b.start for b in bset]
    end_symbols = [b.end for b in bset]
    
    # focus the data
    df = data.iloc[indicies]
    # boundary matching
    phases = []
    cycle_counts = []
    start_indicies = []
    end_indicies = []
    for startidx, row in df.iterrows():
        # continue up until the startidx equals the previous endidx
        if len(end_indicies) > 0:
            if startidx < end_indicies[-1]:
                continue
        start = row['symbol']
        # scan everything in front of the current row
        for endidx, next_row in df.loc[startidx:].iterrows():
            end = next_row['symbol']
            # print(start, end)
            found_end = False
            for b in bset:
                if b.start == start and b.end == end:
                    total_cycles = next_row['cycle'] - row['cycle'] 
                    phases.append(b.name)
                    cycle_counts.append(total_cycles)
                    start_indicies.append(startidx)
                    end_indicies.append(endidx)
                    found_end = True
                    break # on first one, we can stop
            if found_end:
                break
    
    labels = ["phases", "cycle_counts", "start_index", "end_index"]
    res = pd.DataFrame([phases, cycle_counts, start_indicies, end_indicies]).transpose()
    res.columns = labels
    return res

In [4]:
# Smaller bounds should come first.
create = Boundary("create_enclave", "keystone_create_enclave", "keystone_run_enclave")
run_overhead = Boundary("run_enclave", "keystone_run_enclave", "rt_base")

runtime = Boundary("enclave_runtimeuser", "rt_base", "mcall_sm_attest_enclave")
runtime2 = Boundary("enclave_runtimeuser", "return_to_encl", "mcall_sm_stop_enclave")
runtime3 = Boundary("enclave_runtimeuser", "return_to_encl", "mcall_sm_exit_enclave")

attest = Boundary("enclave_attestation", "mcall_sm_attest_enclave", "return_to_encl")
attest_ecall = Boundary("enclave_attestation_edge_call", "mcall_sm_stop_enclave", "return_to_encl")

runtime_noattest = Boundary("enclave_runtimeuser", "rt_base","mcall_sm_stop_enclave")
exit_enclave = Boundary("exit_enclave", "mcall_sm_exit_enclave", "keystone_ioctl")
destroy_bound = Boundary("destroy_enclave", "mcall_sm_destroy_enclave", "enclave_idr_remove")


enclave_bset = [create, run_overhead, runtime, runtime2, runtime3, attest, attest_ecall, runtime_noattest,
               exit_enclave, destroy_bound]

In [5]:
# Run Analysis
def get_buckets(data):
    bset = enclave_bset
    indicies = get_indicies(bset, data)
    # display(data.iloc[indicies]) # sanity check
    phase_table = get_phase_cycles_table(indicies, bset, data)
    # display(phase_table) # sanity check
    results = get_symbol_cost_table(phase_table, data)
    return results.groupby('phase').sum().replace(0, np.nan)

def mean(lst):
    return sum(lst)/len(lst)

def get_error_bounds_per_phase(lst):
    num_phases = lst[0].shape[0]
    
    print("Error bounds:")
    for i in range(num_phases):
        curr_phase_costs = [l.sum(axis=1).values[i] for l in lst]        
        avg = mean(curr_phase_costs)
        mean_sq_diff = [(c - avg)**2 for c in curr_phase_costs]
        avg_mean_sq_diff = mean(mean_sq_diff)
        std_dev = avg_mean_sq_diff**0.5
        err_bound = std_dev / avg
        print(lst[0].index[i] + ":", avg, "std dev:", std_dev, "err:", err_bound * 100)
        
def final_table(lst):
    """Given a lst of bucket tables, get the final result, which is the mean
    of those tables, and sort the indicies by avg value.
    """
    result = mean(lst)
    return result.reindex(result.mean().sort_values().index, axis=1)

In [6]:
t1 = get_buckets(att_trim1)
t2 = get_buckets(att_trim2)
t3 = get_buckets(att_trim3)
t4 = get_buckets(att_trim4)
t5 = get_buckets(att_trim5)

f1 = get_buckets(att_full1)
f2 = get_buckets(att_full2)
f3 = get_buckets(att_full3)
f4 = get_buckets(att_full4)
f5 = get_buckets(att_full5)

a1 = get_buckets(aes1)
a2 = get_buckets(aes2)
a3 = get_buckets(aes3)

atttrimmed = [t1, t2, t3, t4, t5]
attfull = [f1, f2, f3, f4, f5]
aes = [a1, a2, a3]

get_error_bounds_per_phase(atttrimmed)
get_error_bounds_per_phase(attfull)
get_error_bounds_per_phase(aes)

Error bounds:
('create_enclave:', 49494462.2, 'std dev:', 26687.224826871756, 'err:', 0.05391961775245181)
('destroy_enclave:', 240214.4, 'std dev:', 30.017328328817005, 'err:', 0.01249605699276022)
('enclave_attestation:', 1278064.4, 'std dev:', 2218.326269961207, 'err:', 0.17356920902899783)
('enclave_attestation_edge_call:', 4914812.2, 'std dev:', 471.2767339897016, 'err:', 0.009588906245282405)
('enclave_runtimeuser:', 12481.2, 'std dev:', 87.0066664112584, 'err:', 0.69710177235569)
('exit_enclave:', 81846.6, 'std dev:', 1498.771710434915, 'err:', 1.831196055101757)
('run_enclave:', 5212.2, 'std dev:', 172.11670459313353, 'err:', 3.30218918293875)
Error bounds:
('create_enclave:', 49494462.2, 'std dev:', 26687.224826871756, 'err:', 0.05391961775245181)
('destroy_enclave:', 240214.4, 'std dev:', 30.017328328817005, 'err:', 0.01249605699276022)
('enclave_attestation:', 1278064.4, 'std dev:', 2218.326269961207, 'err:', 0.17356920902899783)
('enclave_attestation_edge_call:', 4914812.2,

In [7]:
finalatt= final_table(attfull)
finalatt

,hash_finalize,osm_pmp_set,work_pending,hash_extend,get_enclave_by_id,swake_up,sbi_set_timer,next_event,utm_init,blk_delay_work,...,uart_putchar,sha3_keccakf,___slab_alloc.isra.10.con,__slab_free.isra.14,bio_attempt_back_merge,cfq_allow_bio_merge,elv_bio_merge_ok,finish_fault,ll_back_merge_fn,put_cpu_partial
phase,,,,,,,,,,,,,,,,,,,,,
create_enclave,2.0,NaN,6.2,8.4,NaN,12.0,NaN,20.8,22.0,26.4,...,NaN,48231909.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
destroy_enclave,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
enclave_attestation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,308692.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
enclave_attestation_edge_call,NaN,12.0,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,...,4421833.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
enclave_runtimeuser,NaN,NaN,NaN,NaN,NaN,NaN,18.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
exit_enclave,NaN,6.0,9.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
run_enclave,NaN,6.0,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
finalaes= final_table(aes)
finalaes

,mcall_sm_exit_enclave,hash_finalize,osm_pmp_set,hash_extend,work_pending,get_enclave_by_id,swake_up,finish_swait,next_event,tlb_flush_mmu,...,hash_epm,restore_all,kmem_cache_alloc,__copy_user,init_encl_pgtable,memset,sha3_keccakf,___slab_alloc.isra.10.con,__slab_free.isra.14,put_cpu_partial
phase,,,,,,,,,,,,,,,,,,,,,
create_enclave,NaN,2.0,NaN,6.333333,6.333333,NaN,12.0,20.0,20.0,NaN,...,22992.333333,9705.333333,26628.333333,78413.333333,67819.666667,216673.0,4.823982e+07,NaN,NaN,NaN
destroy_enclave,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,225764.0,NaN,NaN,NaN,NaN
exit_enclave,2.0,NaN,6.0,NaN,9.000000,NaN,NaN,NaN,NaN,26.0,...,NaN,42542.666667,NaN,683.333333,NaN,658.0,NaN,NaN,NaN,NaN
run_enclave,NaN,NaN,6.0,NaN,NaN,10.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

import cufflinks as cf

# You'll need to set API key using:
# plotly.tools.set_credentials_file(username='DemoAccount', api_key='lr1c37zw81')
# Find your api_key here: https://plot.ly/settings/api
# Cufflinks binds plotly to pandas dataframes in IPython notebook.

cf.set_config_file(offline=False, world_readable=True, theme='ggplot')

finalatt.iplot(kind='bar', barmode="stack", filename='name.html')

In [10]:
finalaes.iplot(kind='bar', barmode="stack", filename='name2.html')

In [20]:
# analyze pain points
# for all the phases, get the top offenders
def get_top_x(x, df):
    num_phases = df.shape[0]
    for i in range(num_phases):
        phase_data = df.iloc[[i]].transpose()
        phase = phase_data.columns[0]
        phase_data = phase_data[phase_data[phase].notnull()]
        phase_data.columns=["total_cost"]
        phase_data["total_percent"] = phase_data["total_cost"]/phase_data["total_cost"].sum() * 100
        print(phase)
        print(phase_data.sort_values("total_cost", ascending=False)[:x].to_string())
        print ""

In [21]:
get_top_x(15, finalatt)

create_enclave
                           total_cost  total_percent
sha3_keccakf               48231909.6      97.453451
memset                       216151.6       0.436738
__copy_user                   68160.8       0.137720
init_encl_pgtable             67801.0       0.136993
restore_regs                  34903.2       0.070523
__cpuidle_text_end            30753.8       0.062139
kmem_cache_alloc              24459.0       0.049420
__update_load_avg_cfs_rq.     23483.8       0.047449
hash_epm                      23000.4       0.046473
_raw_spin_unlock_irqresto     22608.8       0.045681
_raw_spin_lock_irqsave        20469.8       0.041360
restore_all                   15496.6       0.031311
kfree                         14794.8       0.029893
trap_vector                   14569.8       0.029439
_raw_spin_lock_irq            14365.6       0.029026

destroy_enclave
                        total_cost  total_percent
memset                    225871.4      94.029084
__free_pages_ok     

In [13]:
get_top_x(15, finalaes)

create_enclave


,total_cost,total_percent
sha3_keccakf,4.823982e+07,97.418231
memset,2.166730e+05,0.437562
__copy_user,7.841333e+04,0.158352
init_encl_pgtable,6.781967e+04,0.136959
restore_regs,3.469967e+04,0.070074
__cpuidle_text_end,3.033567e+04,0.061262
kmem_cache_alloc,2.662833e+04,0.053775
hash_epm,2.299233e+04,0.046432
__update_load_avg_cfs_rq.,2.296967e+04,0.046386
_raw_spin_unlock_irqresto,2.240100e+04,0.045238


destroy_enclave


,total_cost,total_percent
memset,225764.000000,94.004097
__free_pages_ok,5522.000000,2.299262
kfree,1545.333333,0.643449
__mod_zone_page_state,1489.666667,0.620271
pmp_region_free_atomic,1093.666667,0.455383
destroy_enclave,950.000000,0.395563
pmp_unset,622.000000,0.258990
send_and_sync_pmp_ipi,563.333333,0.234562
restore_regs,520.000000,0.216519
free_pages.part.23,514.000000,0.214020


exit_enclave


,total_cost,total_percent
restore_all,42542.666667,43.356762
__mod_node_page_state,3234.666667,3.296565
__cpuidle_text_end,2326.333333,2.370850
_vm_normal_page,1868.333333,1.904086
mark_page_accessed,1864.666667,1.900349
_raw_spin_unlock_irqresto,1798.333333,1.832746
lru_add_drain_cpu,1781.666667,1.815761
trap_vector,1708.000000,1.740684
vmacache_find,1621.666667,1.652699
unmap_page_range,1614.666667,1.645565


run_enclave


,total_cost,total_percent
pmp_set,1172.333333,21.996373
swap_prev_state,848.000000,15.910939
trap_vector,501.000000,9.400213
__radix_tree_lookup,466.333333,8.749765
run_enclave,392.333333,7.361311
__local_bh_enable_ip,392.000000,7.355057
restore_regs,291.000000,5.460004
_raw_spin_lock_bh,274.000000,5.141034
swap_prev_stvec,225.000000,4.221652
get_host_satp,205.000000,3.846394


In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

import pandas as pd
import numpy as np

N = 20
x = np.linspace(1, 10, N)
y = np.random.randn(N)+3
y2 = np.random.randn(N)+6
y3 = np.random.randn(N)+9
y4 = np.random.randn(N)+12
df = pd.DataFrame({'x': x, 'y': y, 'y2':y2, 'y3':y3, 'y4':y4})
df.head()

data = [
    go.Bar(
        x=df['x'], # assign x as the dataframe column 'x'
        y=df['y']
    ),
    go.Bar(
        x=df['x'],
        y=df['y2']
    ),
    go.Bar(
        x=df['x'],
        y=df['y3']
    ),
    go.Bar(
        x=df['x'],
        y=df['y4']
    )

]

layout = go.Layout(
    barmode='stack',
    title='Stacked Bar with Pandas'
)

fig = go.Figure(data=data, layout=layout)

# IPython notebook
py.iplot(fig, filename='pandas-bar-chart-layout')

In [ ]:
# Some notes
# IDEA: Plot top 15 in a pie chart for each binary in each phase (attestation, tiny_aes, max element in array)
# TODO: Remove uart_putchar
# TODO: Automate scripts